In [1]:
import open3d.ml as _ml3d
import open3d as o3d
import open3d.ml.torch as ml3d  # just switch to open3d.ml.tf for tf usage
import os

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
import torch
print(torch.cuda.is_available())

True


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
device

device(type='cuda')

In [6]:
cfg_file = r"/mnt/e/PE/Mathew/Codes/Partitions/Open3D-ML/ml3d/configs/kpconv_parislille3d.yml"
cfg = _ml3d.utils.Config.load_from_file(cfg_file)

model = ml3d.models.KPFCNN(**cfg.model)
cfg.dataset['dataset_path'] = r"/mnt/e/PE/Mathew/Codes/Partitions/Open3D-ML/ml3d/datasets/Paris"
dataset = ml3d.datasets.ParisLille3D(cfg.dataset.pop('dataset_path', None), **cfg.dataset)
pipeline = ml3d.pipelines.SemanticSegmentation(model, dataset=dataset, device="gpu", **cfg.pipeline)

In [7]:
ckpt_folder = "./logs/"
os.makedirs(ckpt_folder, exist_ok=True)
ckpt_path = "ckpt_IIITB.pth"

In [8]:
# load the parameters.
pipeline.load_ckpt(ckpt_path=ckpt_path)

In [9]:
dataset.get_split_list("test")

['/mnt/e/PE/Mathew/Codes/Partitions/Open3D-ML/ml3d/datasets/Paris/test_10_classes/Campus_BlockA_group_1.ply',
 '/mnt/e/PE/Mathew/Codes/Partitions/Open3D-ML/ml3d/datasets/Paris/test_10_classes/Campus_BlockA_group_10.ply',
 '/mnt/e/PE/Mathew/Codes/Partitions/Open3D-ML/ml3d/datasets/Paris/test_10_classes/Campus_BlockA_group_11.ply',
 '/mnt/e/PE/Mathew/Codes/Partitions/Open3D-ML/ml3d/datasets/Paris/test_10_classes/Campus_BlockA_group_12.ply',
 '/mnt/e/PE/Mathew/Codes/Partitions/Open3D-ML/ml3d/datasets/Paris/test_10_classes/Campus_BlockA_group_13.ply',
 '/mnt/e/PE/Mathew/Codes/Partitions/Open3D-ML/ml3d/datasets/Paris/test_10_classes/Campus_BlockA_group_14.ply',
 '/mnt/e/PE/Mathew/Codes/Partitions/Open3D-ML/ml3d/datasets/Paris/test_10_classes/Campus_BlockA_group_15.ply',
 '/mnt/e/PE/Mathew/Codes/Partitions/Open3D-ML/ml3d/datasets/Paris/test_10_classes/Campus_BlockA_group_16.ply',
 '/mnt/e/PE/Mathew/Codes/Partitions/Open3D-ML/ml3d/datasets/Paris/test_10_classes/Campus_BlockA_group_17.ply',
 '

In [11]:
test_split = dataset.get_split("test")
len(test_split)
# data = test_split.get_data(2)

4

In [32]:
# run inference on a single example.
# returns dict with 'predict_labels' and 'predict_scores'.
result = pipeline.run_inference(data)


test 0/1: 100%|███████████████████████████████████████████████████████████████| 443889/443889 [06:06<00:00, 1210.30it/s]

test 0/1: 100%|█████████████████████████████████████████████████████████████| 1571935/1571935 [07:15<00:00, 1375.93it/s]

In [28]:
result.keys()

dict_keys(['predict_labels', 'predict_scores'])

In [27]:
len(result['predict_labels'])

2275320

In [18]:
set(result['predict_labels'])

{0, 1, 2, 4, 5, 6, 7, 8}

In [16]:
import numpy as np
import matplotlib.pyplot as plt

points = data["point"]
labels = result["predict_labels"]

colours = plt.get_cmap("tab20")(labels / labels.max())[:, :3]

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points)
pcd.colors = o3d.utility.Vector3dVector(colours)

o3d.io.write_point_cloud("inference_result.ply", pcd)
print("Done")

Done


In [34]:
import numpy as np

points = data["point"]
labels = result["predict_labels"]
# scores = result["predict_scores"]

# vertex = np.array(
#     [tuple(p) + (int(l),) for p, l in zip(points, labels)],
#     dtype=[("x", "f4"), ("y", "f4"), ("z", "f4"), ("label", "i4")]
# )

# ply = PlyData([PlyElement.describe(vertex, "vertex")], text=True)

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points)
pcd.colors = o3d.utility.Vector3dVector(np.zeros_like(points))  # optional placeholder

with open("test_result_dijon_9_2.ply", "w") as f:
    f.write("ply\n")
    f.write("format ascii 1.0\n")
    f.write("element vertex {}\n".format(len(points)))
    f.write("property float x\n")
    f.write("property float y\n")
    f.write("property float z\n")
    # f.write("property uchar red\n")
    # f.write("property uchar green\n")
    # f.write("property uchar blue\n")
    f.write("property float scalar_Classification\n")
    # f.write("property float intensity\n")
    f.write("end_header\n")

    for p, l in zip(points, labels):
        f.write(f"{p[0]} {p[1]} {p[2]} {float(l)}\n")

print("Done")

Done


In [14]:
newData = data

In [15]:
newData['label'] = result['predict_labels']

In [16]:
newData

{'point': array([[-1011.72906 , -1235.7638  ,    25.576008],
        [-1005.6605  , -1224.2548  ,    25.378702],
        [-1011.5628  , -1235.7372  ,    25.638912],
        ...,
        [-1082.7506  , -1143.0483  ,    46.396255],
        [-1082.3799  , -1143.3535  ,    46.463024],
        [-1085.1509  , -1139.7754  ,    45.107265]], dtype=float32),
 'feat': None,
 'label': array([8, 1, 8, ..., 1, 1, 1])}

In [17]:
# evaluate performance on the test set; this will write logs to './logs'.
pipeline.run_test()


preprocess: 100%|█████████████████████████████████████████████████████████████████████████| 3/3 [01:15<00:00, 25.20s/it]

test 0/1: 100%|█████████████████████████████████████████████████████████████| 1905227/1905227 [10:46<00:00, 2945.45it/s]

test 1/3: 100%|█████████████████████████████████████████████████████████████| 1674722/1674722 [05:43<00:00, 4877.52it/s]

test 2/3: 100%|█████████████████████████████████████████████████████████████| 1571935/1571935 [03:16<00:00, 3459.99it/s]

TypeError: 'NoneType' object is not subscriptable

## Raw labelling

In [37]:
import numpy as np

# Assuming newData is already defined as in your example
newData['label'] = newData['label'] + 1

# Combine all data into a single array for writing to a text file
# Structure: x, y, z, r, g, b, label
data_to_write = np.hstack((
    newData['point'],                       # x, y, z coordinates
    #newData['feat'][:, :3],                 # r, g, b colors
    newData['label'].reshape(-1, 1)         # label as a single column
))

# Save to a text file ckpt_00300
np.savetxt("IIITB_Pred_PE.txt", data_to_write, fmt='%.6f %.6f %.6f %d')

print("Point cloud data with labels saved to 1000epoch.txt")

Point cloud data with labels saved to 1000epoch.txt


In [38]:
import numpy as np

# Extract point cloud data: x, y, z, label
labels = newData['label'].reshape(-1, 1).astype(np.float32)
data_to_write = np.hstack((newData['point'], labels))

# Create a modified header for the PCD file (without RGB)
header = f"""# .PCD v0.7 - Point Cloud Data file format
VERSION 0.7
FIELDS x y z label
SIZE 4 4 4 4
TYPE F F F F
COUNT 1 1 1 1
WIDTH {data_to_write.shape[0]}
HEIGHT 1
VIEWPOINT 0 0 0 1 0 0 0
POINTS {data_to_write.shape[0]}
DATA ascii
"""

# Combine points, packed RGB, and labels
pcd_data = np.hstack((newData['point'], labels))

# Write to the PCD file
with open("IIITB_Pred_PE.pcd", "w") as f:
    f.write(header)
    np.savetxt(f, data_to_write, fmt="%.6f %.6f %.6f %.1f")

# print("Point cloud with labels saved to ParisLilleOut_Labeled.pcd")